In [2]:
import requests
import os 
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [ ]:
service_key = os.getenv('service_key')
url = 'https://apis.data.go.kr/1741000/RegistrationPopulationByRegion'